## FACT ORDERS
**Data Reading**

In [0]:
# IMPORTS
from pyspark.sql.functions import *

In [0]:
df = spark.table("db_dlt_proj.silver.orders")

### Fact Dataframe

In [0]:
dim_cust = spark.table("db_dlt_proj.gold.dim_customers").select(col("DImCustomerKey"),col("customer_id").alias("dim_customer_id"))
dim_prod = spark.table("db_dlt_proj.gold.dim_products").select(col("product_id").alias("DimProductKey"),col("product_id").alias("dim_product_id"))

In [0]:
df_fact = df.alias('orders').join(dim_cust.alias('cust'),col('orders.customer_id')==col('cust.dim_customer_id'),'left')\
                            .join(dim_prod.alias('prod'),col('orders.product_id')==col('prod.dim_product_id'),'left')

df_fact = df_fact.drop('dim_customer_id','dim_product_id','customer_id','product_id')
# df_fact.show()

# UPSERT ON FACT

In [0]:
from delta.tables import DeltaTable

if spark.catalog.tableExists("db_dlt_proj.gold.orders_fact"):
  target = DeltaTable.forName(spark,"db_dlt_proj.gold.orders_fact")
  (target.alias("target")\
    .merge(
      df_fact.alias("updates"),
      "updates.order_id = target.order_id AND updates.DimProductKey = target.DimProductKey AND updates.DImCustomerKey = target.DImCustomerKey")\
  .whenMatchedUpdateAll()\
  .whenNotMatchedInsertAll()\
  .execute())
else:
  df_fact.write.format('delta')\
          .option('path', "/Volumes/db_dlt_proj/ext_location/gold_external_location/orders_fact/")\
          .save()
  df_fact.write.mode("overwrite").saveAsTable("db_dlt_proj.gold.orders_fact")

In [0]:
%sql
select count(*) from db_dlt_proj.gold.orders_fact;